In [4]:
from sklearn import svm
import numpy as np
import os
import numpy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import collections
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
def SVM(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)

# Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	

    
# SVM Classifier Valence
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
#     print(cm)
#     print("Accuracy score of Valence ")
#     print(accuracy_score(y_test, y_predict)*100)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    	
# SVM Classifier Arousal
    clf = SVC(kernel = 'poly')
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    cm = confusion_matrix(z_test, z_predict)
#     print(z_test)
#     print(z_predict)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100]

    

def SVMCombined(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    return [accuracy_score(y_test, y_predict)*100]



def SVMHS(UserName,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
        
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)  	
    clf = SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    cm = confusion_matrix(y_test, y_predict)
    print(f1_score(y_test, y_predict, average='weighted'))
    return [accuracy_score(y_test, y_predict)*100]

def Run100(UserName,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    for i in range (100):
        svmpca=SVM(UserName,Window,StartTime,EndTime)
        arousal+=[svmpca[0]]
        valence+=[svmpca[1]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence)];
    

    
def RunCombined100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMCombined(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Combined "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined)
    
def RunHS100(UserName,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    for i in range (100):
        svm=SVMHS(UserName,Window,StartTime,EndTime)
        Combined+=svm
    #print("Happy and Sad "+str(np.mean(Combined)))
    print(collections.Counter(Combined))
    return np.mean(Combined);

    
if __name__ == "__main__":
    HS=0;
    Combined=0;
    Valence=0
    Arousal=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HS+=var
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        Combined+=var1
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var1)
        Valence+=var2[0]
        Arousal+=var2[1]
    print("Average happy sad for first 30 "+str(HS/30))
    print("Average Combined sad for first 30 "+str(Combined/30))
    print("Average Valence for first 30 "+str(Valence/30))
    print("Average Arousal for first 30 "+str(Arousal/30))

s01
0.06666666666666668
0.33333333333333337
0.4499999999999999
0.0
0.2
0.2285714285714286
0.34285714285714286
0.4499999999999999
0.4499999999999999
0.7111111111111111
0.2285714285714286
0.7111111111111111
0.4499999999999999
0.6
0.4499999999999999
0.4499999999999999
0.2285714285714286
0.0
0.6333333333333333
0.4499999999999999
0.4571428571428572
0.4499999999999999
0.4499999999999999
0.6
0.2285714285714286
0.0
0.4499999999999999
0.2285714285714286
0.34285714285714286
0.2
0.4499999999999999
0.2285714285714286
0.34285714285714286
0.4499999999999999
0.4499999999999999
0.4571428571428572
0.2285714285714286
0.06666666666666668
0.2285714285714286
0.06666666666666668
0.4499999999999999
0.4499999999999999
0.2285714285714286
0.7111111111111111
0.06666666666666668
0.4499999999999999
0.2285714285714286
0.4499999999999999
0.4499999999999999
0.26666666666666666
0.2285714285714286
0.2285714285714286
0.4499999999999999
0.6
0.6
0.2285714285714286
0.2285714285714286
0.7499999999999999
0.2285714285714286
0

0.5333333333333333
0.7575757575757575
0.7575757575757575
0.3333333333333333
1.0
0.5333333333333333
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.7575757575757575
1.0
0.7575757575757575
0.7575757575757575
0.3333333333333333
0.5333333333333333
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.7575757575757575
0.5333333333333333
0.3333333333333333
0.7575757575757575
1.0
0.5333333333333333
0.7575757575757575
0.5333333333333333
0.7575757575757575
0.3333333333333333
1.0
0.5333333333333333
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.7575757575757575
0.7575757575757575
0.3333333333333333
1.0
0.7575757575757575
0.5333333333333333
0.7575757575757575
0.7575757575757575
0.7575757575757575
1.0
0.7575757575757575
0.7575757575757575
0.7575757575757575
1.0
0.7575757575757575
0.7575757575757575
0.7575757575757575
0.7575757575757575
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.16666666666666666
0.

0.3333333333333333
1.0
0.3333333333333333
0.8148148148148149
0.16666666666666666
0.851851851851852
0.9090909090909091
0.6249999999999999
0.6666666666666669
0.8148148148148149
0.8148148148148149
0.6249999999999999
0.16666666666666666
0.7575757575757575
0.5555555555555555
0.8000000000000002
0.5333333333333333
0.7083333333333334
0.5333333333333333
0.3333333333333333
0.851851851851852
0.5333333333333333
Counter({66.66666666666666: 43, 83.33333333333334: 31, 50.0: 15, 100.0: 5, 33.33333333333333: 5, 16.666666666666664: 1})
Happy Sad
68.83333333333333
Counter({50.0: 32, 37.5: 25, 62.5: 21, 25.0: 11, 75.0: 5, 12.5: 5, 0.0: 1})
Combined Emotions
45.625
Arousal 68.875
Counter({62.5: 31, 75.0: 23, 87.5: 22, 50.0: 14, 37.5: 4, 100.0: 4, 25.0: 2})
Valence 57.25
Counter({62.5: 27, 50.0: 26, 37.5: 18, 75.0: 17, 87.5: 8, 25.0: 3, 12.5: 1})
Arousal Valence
45.625
s10
0.8571428571428571
0.6428571428571428
0.6428571428571428
0.6428571428571428
0.6428571428571428
0.6428571428571428
0.3333333333333333
0.3

0.3333333333333333
1.0
0.6428571428571428
1.0
0.3333333333333333
0.6428571428571428
0.3333333333333333
0.6428571428571428
0.6428571428571428
1.0
Counter({75.0: 41, 50.0: 30, 100.0: 22, 25.0: 7})
Happy Sad
69.5
Counter({50.0: 28, 37.5: 28, 62.5: 19, 75.0: 9, 25.0: 8, 12.5: 7, 0.0: 1})
Combined Emotions
46.0
Arousal 68.0
Counter({62.5: 32, 75.0: 30, 87.5: 19, 50.0: 11, 37.5: 6, 12.5: 1, 100.0: 1})
Valence 67.625
Counter({75.0: 28, 62.5: 25, 87.5: 18, 50.0: 16, 37.5: 8, 100.0: 4, 25.0: 1})
Arousal Valence
46.0
s15
0.6428571428571428
0.0
0.30000000000000004
0.3333333333333333
0.8571428571428571
0.6428571428571428
0.1
0.6428571428571428
0.30000000000000004
0.3333333333333333
0.6428571428571428
0.1
0.4
0.3333333333333333
0.6428571428571428
0.3333333333333333
0.3333333333333333
0.3333333333333333
0.5
0.3333333333333333
0.3333333333333333
0.4
0.3333333333333333
0.3333333333333333
0.0
0.3333333333333333
0.3333333333333333
0.5
0.4
0.8571428571428571
0.4
0.6428571428571428
0.3333333333333333
0.33

0.3333333333333333
1.0
0.3333333333333333
0.3333333333333333
0.5333333333333333
0.6666666666666666
0.5333333333333333
0.7575757575757575
0.3333333333333333
0.7575757575757575
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.7575757575757575
0.3333333333333333
0.5333333333333333
0.9090909090909091
0.7575757575757575
0.5333333333333333
0.3333333333333333
1.0
0.7575757575757575
0.7575757575757575
0.5333333333333333
0.8000000000000002
0.5333333333333333
0.5333333333333333
0.3333333333333333
1.0
0.7575757575757575
0.7575757575757575
1.0
0.7575757575757575
1.0
0.3333333333333333
0.7575757575757575
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5333333333333333
0.5555555555555555
0.5333333333333333
0.7575757575757575
0.8000000000000002
0.7575757575757575
0.5333333333333333
0.7575757575757575
0.7575757575757575
1.0
0.5333333333333333
0.8000000000000002
0.7575757575757575
0.7575757575757575
0.3333333333333333
0.8000000000000002
1.0
0.7575757575757575
0.5

1.0
here
here
here
here
1.0
here
here
1.0
here
1.0
1.0
1.0
1.0
1.0
1.0
here
here
1.0
1.0
1.0
1.0
1.0
Counter({100.0: 100})
Happy Sad
100.0
Counter({50.0: 34, 37.5: 26, 62.5: 20, 25.0: 11, 12.5: 5, 75.0: 3, 87.5: 1})
Combined Emotions
45.75
Arousal 88.625
Counter({100.0: 38, 87.5: 36, 75.0: 23, 62.5: 3})
Valence 39.125
Counter({37.5: 32, 50.0: 28, 25.0: 22, 62.5: 9, 12.5: 4, 0.0: 3, 75.0: 2})
Arousal Valence
45.75
s25
1.0
0.6428571428571428
0.6428571428571428
1.0
0.6428571428571428
0.6428571428571428
1.0
1.0
1.0
0.6428571428571428
1.0
1.0
1.0
0.6428571428571428
1.0
1.0
1.0
0.6428571428571428
0.6428571428571428
1.0
0.6428571428571428
1.0
0.6428571428571428
here
1.0
0.6428571428571428
0.6428571428571428
0.6428571428571428
1.0
1.0
0.6428571428571428
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6428571428571428
1.0
here
1.0
0.6428571428571428
1.0
1.0
0.6428571428571428
0.6428571428571428
1.0
0.6428571428571428
0.6428571428571428
here
1.0
1.0
1.0
1.0
1.0
0.6428571428571428
1.0
0.6428571428571428
0.64285714

0.5333333333333333
1.0
0.6249999999999999
0.5333333333333333
0.5333333333333333
0.6249999999999999
0.6249999999999999
0.16666666666666666
0.3333333333333333
0.8148148148148149
0.8148148148148149
0.5333333333333333
0.6249999999999999
0.8148148148148149
0.6249999999999999
0.5333333333333333
0.3333333333333333
0.5333333333333333
0.5333333333333333
0.6249999999999999
1.0
0.3333333333333333
0.9090909090909091
0.3333333333333333
0.5333333333333333
0.5333333333333333
0.8148148148148149
0.8148148148148149
0.7575757575757575
1.0
0.6249999999999999
0.3333333333333333
0.3333333333333333
0.6666666666666666
0.3333333333333333
0.8148148148148149
1.0
0.6249999999999999
0.5428571428571429
0.8148148148148149
0.3333333333333333
0.7575757575757575
1.0
0.6249999999999999
0.5333333333333333
0.7575757575757575
0.8148148148148149
0.5333333333333333
1.0
0.047619047619047616
0.5333333333333333
0.7083333333333334
1.0
0.7575757575757575
0.7575757575757575
0.8148148148148149
0.5333333333333333
1.0
0.5333333333333